In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, max, min, when, rank, sum as _sum, current_date, datediff
from pyspark.sql.window import Window
from pyspark.sql.functions import lit, expr
import random
from datetime import datetime, timedelta

spark = SparkSession.builder.appName("AdvancedEmployeeData").getOrCreate()

# Project dataset
project_data = [
    ("Ananya", "HR Portal", 120),
    ("Rahul", "Data Platform", 200),
    ("Priya", "Data Platform", 180),
    ("Zoya", "Campaign Tracker", 100),
    ("Karan", "HR Portal", 130),
    ("Naveen", "ML Pipeline", 220),
    ("Fatima", "Campaign Tracker", 90)
]
columns_proj = ["Name", "Project", "HoursWorked"]
df_proj = spark.createDataFrame(project_data, columns_proj)

# Recreate employee_data
data = [
    ("Ananya", "HR", 52000),
    ("Rahul", "Engineering", 65000),
    ("Priya", "Engineering", 60000),
    ("Zoya", "Marketing", 48000),
    ("Karan", "HR", 53000),
    ("Naveen", "Engineering", 70000),
    ("Fatima", "Marketing", 45000)
]
columns = ["Name", "Department", "Salary"]
df_emp = spark.createDataFrame(data, columns)

# Recreate performance_data
performance = [
    ("Ananya", 2023, 4.5),
    ("Rahul", 2023, 4.9),
    ("Priya", 2023, 4.3),
    ("Zoya", 2023, 3.8),
    ("Karan", 2023, 4.1),
    ("Naveen", 2023, 4.7),
    ("Fatima", 2023, 3.9)
]
columns_perf = ["Name", "Year", "Rating"]
df_perf = spark.createDataFrame(performance, columns_perf)

# Recreate project_data
project_data = [
    ("Ananya", "HR Portal", 120),
    ("Rahul", "Data Platform", 200),
    ("Priya", "Data Platform", 180),
    ("Zoya", "Campaign Tracker", 100),
    ("Karan", "HR Portal", 130),
    ("Naveen", "ML Pipeline", 220),
    ("Fatima", "Campaign Tracker", 90)
]
columns_proj = ["Name", "Project", "HoursWorked"]
df_proj = spark.createDataFrame(project_data, columns_proj)



In [4]:
# 1. Join all three datasets on Name
df_all = df_emp.join(df_perf, on="Name", how="inner") \
               .join(df_proj, on="Name", how="inner")
df_all.show()


+------+-----------+------+----+------+----------------+-----------+
|  Name| Department|Salary|Year|Rating|         Project|HoursWorked|
+------+-----------+------+----+------+----------------+-----------+
|Ananya|         HR| 52000|2023|   4.5|       HR Portal|        120|
| Priya|Engineering| 60000|2023|   4.3|   Data Platform|        180|
| Rahul|Engineering| 65000|2023|   4.9|   Data Platform|        200|
|Naveen|Engineering| 70000|2023|   4.7|     ML Pipeline|        220|
|Fatima|  Marketing| 45000|2023|   3.9|Campaign Tracker|         90|
|  Zoya|  Marketing| 48000|2023|   3.8|Campaign Tracker|        100|
| Karan|         HR| 53000|2023|   4.1|       HR Portal|        130|
+------+-----------+------+----+------+----------------+-----------+



In [5]:
# 2. Total hours worked per department
df_all.groupBy("Department").sum("HoursWorked").withColumnRenamed("sum(HoursWorked)", "TotalHours").show()


+-----------+----------+
| Department|TotalHours|
+-----------+----------+
|Engineering|       600|
|         HR|       250|
|  Marketing|       190|
+-----------+----------+



In [6]:
# 3. Average rating per project
df_all.groupBy("Project").avg("Rating").withColumnRenamed("avg(Rating)", "AvgRating").show()

+----------------+------------------+
|         Project|         AvgRating|
+----------------+------------------+
|   Data Platform|               4.6|
|       HR Portal|               4.3|
|     ML Pipeline|               4.7|
|Campaign Tracker|3.8499999999999996|
+----------------+------------------+



In [7]:
#  4. Add a row with a None rating
from pyspark.sql import Row

new_row = Row("Meena", 2023, None)
df_perf_null = df_perf.union(spark.createDataFrame([new_row], df_perf.schema))
df_perf_null.show()

+------+----+------+
|  Name|Year|Rating|
+------+----+------+
|Ananya|2023|   4.5|
| Rahul|2023|   4.9|
| Priya|2023|   4.3|
|  Zoya|2023|   3.8|
| Karan|2023|   4.1|
|Naveen|2023|   4.7|
|Fatima|2023|   3.9|
| Meena|2023|  NULL|
+------+----+------+



In [8]:
 # 5. Filter rows with null values
 df_perf_null.filter(col("Rating").isNull()).show()

+-----+----+------+
| Name|Year|Rating|
+-----+----+------+
|Meena|2023|  NULL|
+-----+----+------+



In [9]:
# 6. Replace null ratings with department average
df_perf_null_joined = df_perf_null.join(df_emp, on="Name", how="left")

In [10]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, when

windowSpec = Window.partitionBy("Department")
df_filled = df_perf_null_joined.withColumn(
    "FilledRating",
    when(col("Rating").isNull(), avg("Rating").over(windowSpec)).otherwise(col("Rating"))
)
df_filled.select("Name", "Department", "Rating", "FilledRating").show()

+------+-----------+------+------------+
|  Name| Department|Rating|FilledRating|
+------+-----------+------+------------+
| Meena|       NULL|  NULL|        NULL|
| Priya|Engineering|   4.3|         4.3|
| Rahul|Engineering|   4.9|         4.9|
|Naveen|Engineering|   4.7|         4.7|
|Ananya|         HR|   4.5|         4.5|
| Karan|         HR|   4.1|         4.1|
|Fatima|  Marketing|   3.9|         3.9|
|  Zoya|  Marketing|   3.8|         3.8|
+------+-----------+------+------------+



In [11]:
# 7. Create PerformanceCategory column
df_all = df_all.withColumn("PerformanceCategory",
    when(col("Rating") >= 4.7, "Excellent")
    .when((col("Rating") >= 4.0) & (col("Rating") < 4.7), "Good")
    .otherwise("Average")
)
df_all.select("Name", "Rating", "PerformanceCategory").show()

+------+------+-------------------+
|  Name|Rating|PerformanceCategory|
+------+------+-------------------+
|Ananya|   4.5|               Good|
| Priya|   4.3|               Good|
| Rahul|   4.9|          Excellent|
|Naveen|   4.7|          Excellent|
|Fatima|   3.9|            Average|
|  Zoya|   3.8|            Average|
| Karan|   4.1|               Good|
+------+------+-------------------+



In [12]:
#  8. Create UDF to assign bonus
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def bonus_calc(hours):
    return 10000 if hours > 200 else 5000

bonus_udf = udf(bonus_calc, IntegerType())

df_all = df_all.withColumn("Bonus", bonus_udf(col("HoursWorked")))
df_all.select("Name", "HoursWorked", "Bonus").show()

+------+-----------+-----+
|  Name|HoursWorked|Bonus|
+------+-----------+-----+
|Ananya|        120| 5000|
| Priya|        180| 5000|
| Rahul|        200| 5000|
|Naveen|        220|10000|
|Fatima|         90| 5000|
|  Zoya|        100| 5000|
| Karan|        130| 5000|
+------+-----------+-----+



In [13]:
#  9. Add JoinDate and MonthsWorked
from pyspark.sql.functions import to_date, months_between, current_date

df_all = df_all.withColumn("JoinDate", to_date(lit("2021-06-01")))
df_all = df_all.withColumn("MonthsWorked", months_between(current_date(), col("JoinDate")).cast("int"))
df_all.select("Name", "JoinDate", "MonthsWorked").show()

+------+----------+------------+
|  Name|  JoinDate|MonthsWorked|
+------+----------+------------+
|Ananya|2021-06-01|          48|
| Priya|2021-06-01|          48|
| Rahul|2021-06-01|          48|
|Naveen|2021-06-01|          48|
|Fatima|2021-06-01|          48|
|  Zoya|2021-06-01|          48|
| Karan|2021-06-01|          48|
+------+----------+------------+



In [14]:
# 10. Count employees who joined before 2022
df_all.filter(col("JoinDate") < to_date(lit("2022-01-01"))).select("Name").distinct().count()

7

In [15]:
# 11. Union with extra team members
extra_employees = [
    ("Meena", "HR", 48000),
    ("Raj", "Marketing", 51000)
]
df_extra = spark.createDataFrame(extra_employees, ["Name", "Department", "Salary"])
df_combined = df_emp.union(df_extra)
df_combined.show()


+------+-----------+------+
|  Name| Department|Salary|
+------+-----------+------+
|Ananya|         HR| 52000|
| Rahul|Engineering| 65000|
| Priya|Engineering| 60000|
|  Zoya|  Marketing| 48000|
| Karan|         HR| 53000|
|Naveen|Engineering| 70000|
|Fatima|  Marketing| 45000|
| Meena|         HR| 48000|
|   Raj|  Marketing| 51000|
+------+-----------+------+



In [16]:
# 12. Save final merged dataset partitioned by department
df_all.write.mode("overwrite").partitionBy("Department").parquet("/content/final_employee_dataset_partitioned")